<a href="https://colab.research.google.com/github/arssite/Drowsiness/blob/main/Detection_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
import time
import pygame

# Load the trained model
model = load_model('Driver_Drowsiness_Detection.h5')

# Initialize pygame for sound alert
pygame.mixer.init()

# Load the cascade classifiers for face and eye detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Function to detect drowsiness
def detect_drowsiness(frame, model):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    res = 0  # Initialize res variable
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)

        for (ex, ey, ew, eh) in eyes:
            eye_roi = roi_color[ey:ey + eh, ex:ex + ew]
            eye_roi_color = cv2.cvtColor(eye_roi, cv2.COLOR_BGR2RGB)  # Convert grayscale to color
            resized = cv2.resize(eye_roi_color, (32, 32))  # Resize the image to (32, 32)
            normalized = resized / 255.0
            reshaped = np.reshape(normalized, (-1, 32, 32, 3))  # Reshape to match model input shape
            result = model.predict(reshaped)
            print("Result", result[0][1])
            res = result[0][1]
            if result[0][1] > 0.01:
                return True, res  # Drowsiness detected

    return False, res  # No drowsiness detected

# Main function
def main():
    cap = cv2.VideoCapture(0)

    alert_triggered = False
    drowsy_start_time = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        drowsy, result = detect_drowsiness(frame, model)
        if not drowsy:
            if drowsy_start_time is None:
                drowsy_start_time = time.time()
                print("Drowsiness detected. Waiting for 2 seconds before playing alert sound...")
            elif time.time() - drowsy_start_time > 1:
                if not alert_triggered:
                    print("Playing alert sound...")
                    pygame.mixer.music.load('beep-warning-6387.mp3')  # Load alert sound
                    pygame.mixer.music.play(-1)  # Play alert sound in a loop
                    alert_triggered = True
        else:
            if alert_triggered:
                print("Stopping alert sound...")
                pygame.mixer.music.stop()  # Stop playing alert sound
                alert_triggered = False
            drowsy_start_time = None

        cv2.putText(frame, f"State: {'Drowsy' if  not drowsy else 'Alert'}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.imshow('Drowsiness Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()